In [2]:
import signal
import requests
from time import sleep
import numpy as np
import sys

class ApiException(Exception):
    pass

def signal_handler(sigum, frame):
    global shutdown 
    signal.signal(signal.SIGINT, signal.SIG_DFL)
    shutdown=True
    
API_KEY = {'X-API-Key': 'YA6YJ9TB'}
shutdown=False

s = requests.session() 
s.headers.update(API_KEY)

#Global Variables
SPEEDBUMP = 0.5
MAX_LONG_EXPOSURE_NET = 25000
MAX_SHORT_EXPOSURE_NET = -25000
MAX_EXPOSURE_GROSS = 500000
ORDER_LIMIT = 5000
MAX_ORDERS = 5
ticker_list = ['CNR', 'RY', 'AC']

def get_tick(s):   
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']
    raise ApiException('Authorization error, check key')


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position():
    resp = s.get('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        gross_position = abs(book[0]['position']) + abs(book[1]['position']) + abs(book[2]['position'])
        net_position = book[0]['position'] + book[1]['position'] + book[2]['position']
        return gross_position, net_position

def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders
    
def open_sells(s):
    resp = s.get('http://localhost:9999/v1/orders?status=OPEN')
    if resp.ok:
        open_sells_volume = 0  #total combined volume of all open sells
        ids = []  #all open sell ids
        prices = []  #all open sell prices
        order_volumes = []  #all open sell volumes
        volume_filled = []  #volume filled for each open sell order
     
        open_orders = resp.json()
        for order in open_orders:
            if order['action'] == 'SELL':
                volume_filled.append(order['quantity_filled'])
                order_volumes.append(order['quantity'])
                open_sells_volume = open_sells_volume + order['quantity']
                prices.append(order['price'])
                ids.append(order['order_id'])
                
        return volume_filled, open_sells_volume, ids, prices, order_volumes

def open_buys(s):
    resp=s.get('http://localhost:9999/v1/orders?status=OPEN')
    if resp.ok:
        open_buys_volume = 0 
        ids = []
        prices = [] 
        order_volumes = []
        volume_filled = []
         
        open_orders = resp.json()
        for order in open_orders:
            if order[ 'action'] == 'BUY':
                open_buys_volume = open_buys_volume + order['quantity']
                volume_filled.append(order['quantity_filled'])
                order_volumes.append(order['quantity'])
                prices.append(order['price'])
                ids.append(order['order_id'])
     
        return volume_filled, open_buys_volume, ids, prices, order_volumes

def buy_sell(s, sell_price, buy_price, ticker_symbol):
    for i in range(MAX_ORDERS):
        s.post('http://localhost:9999/v1/orders',params={'ticker': ticker_symbol, 'type': 'LIMIT',
        'quantity': ORDER_LIMIT, 'price': sell_price, 'action': 'SELL'})
        s.post('http://localhost:9999/v1/orders', params={'ticker': ticker_symbol, 'type': 'LIMIT', 
        'quantity': ORDER_LIMIT, 'price': buy_price, 'action': 'BUY'})
        
def buy_sell(s, sell_price, buy_price, ticker_symbol):
    for i in range(MAX_ORDERS):
        s.post('http://localhost:9999/v1/orders',params={'ticker': ticker_symbol, 'type': 'LIMIT',
        'quantity': ORDER_LIMIT, 'price': sell_price, 'action': 'SELL'})
        s.post('http://localhost:9999/v1/orders', params={'ticker': ticker_symbol, 'type': 'LIMIT', 
        'quantity': ORDER_LIMIT, 'price': buy_price, 'action': 'BUY'})
        
def re_order(s, number_of_orders, ids, volumes_filled, volumes, price, action):
    for i in range(number_of_orders):
        id = ids[i]
        volume = volumes[i]
        volume_filled = volumes_filled[i]
        #if the order is partially filled.
        if(volume_filled != 0):
            volume = ORDER_LIMIT - volume_filled
            #delete then re-purcnase.
            deleted = s.delete('http://localhost:9999/v1/orders/{)'.format(id))
            if(deleted.ok):
                s.post('http://localhost:9999/v1/orders', params ={'ticker': ticker_symbol,
                'type': 'LIMIT', 'quantity': volume, 'price': price, 'action': action})

def get_order_status(order_id):
    resp = s.get('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']

def main():
    #instantiate variables about all the open buy orders
    buy_ids = []
    buy_prices = []
    buy_volumes = []
    volume_filled_buys = []
    open_buys_volume = 0
     
    #instantiate variables about all the open sell orders
    sell_ids = []
    sell_prices = []
    sell_volumes = []
    volume_filled_sells = []
    open_sells_volume = 0
     
    #instantiated variables when just one side of the book has been completely filled
    single_side_filled = False
    single_side_transaction_time = 0
    #creates a session to manage connections and requests to the ril client
    tick, status = get_tick(s)
    ticker_list = ['CNR', 'RY', 'AC']
    market_prices = np.array([0.,0.,0.,0.,0.,0.])
    market_prices = market_prices.reshape(3,2)
 
    while (tick > 5 and tick < 295 and not shutdown) and status == 'ACTIVE':
        for i in range(3):
            ticker_symbol = ticker_list[i]
                     
 
            gross_position, net_position = get_position()
            fee = 0.01
               
            #undate intormation about the case
            volume_filled_sells, open_sells_volume, sell_ids, sell_prices, sell_volumes = open_sells(s)
            volume_filled_buys, open_buys_volume, buy_ids, buy_prices, buy_volumes = open_buys(s)
             
             
            #check for 0 orders
            if(open_sells_volume == 0 and open_buys_volume == 0):
                #both sides are filled now
                single_side_filled = False
                
                bid_price, ask_price = get_bid_ask(ticker_symbol) 
             
                #calculate the spread between the bid and ask prices
                bid_ask_spread = ask_price - bid_price
                 
                #set the prices
                sell_price = ask_price
                buy_price = bid_price
                 
                #the calculated spread is greater or equal to our set spread
                if(bid_ask_spread >= fee):
                    #buy and sell the maximum number of shares
                    buy_sell(s, sell_price, buy_price, ticker_symbol)
                    sleep(SPEEDBUMP)
     
            #there are oustanding open orders
            else:
                #one side of the book nas no open orders 
                if(not single_side_filled and (open_buys_volume == 0 or open_sells_volume == 0)):
                    single_side_filled = True
                    single_side_transaction_time = tick
                 
                #ask side has been completely filled
                if(open_sells_volume == 0):
                    #current buy orders are at the top of the book
                    if(buy_price == bid_price):
                        continue  #next iteration of Loop
             
                    #its been more than 3 seconds since a single Side has been completely Tilled
                    elif(tick - single_side_transaction_time >= 3):
                        #calulate the potential profits you can make
                        next_buy_price = bid_price
                        potential_profit = sell_price - next_buy_price - (2 * fee)
 
                        #potential protit is greater than or equal to a cent or its been more than 6 seconds
                        if(potential_profit >= fee or tick - single_side_transaction_time >= 6):
                            action = "BUY"
                            number_of_orders = len(buy_ids)
                            buy_price = bid_price + fee
                            price = buy_price
                            ids = buy_ids
                            volumes = buy_volumes
                            volumes_filled = volume_filled_buys
                     
                            #delete buys and re-buy
                            re_order(s, number_of_orders, ids, volumes_filled, volumes, price, action)
                            sleep(SPEEDBUMP)
         
                #bid side has been completely filled
                elif (open_buys_volume == 0):
                    #current sell orders are at the top of the book
                    if(sell_price == ask_price):
                        continue  #next iteration of loop
             
                    #its been more than 3 seconds since a single side has been completely filled
                    elif(tick - single_side_transaction_time >= 3):
                        #calculate the potential protit you can make
                        next_sell_price = ask_price
                        potential_profit =  next_sell_price - buy_price - (2 * fee)
                 
                        #potential profit is greater than or equal to a cent or its been more than 6 seconds
                        if(potential_profit >= fee or tick - single_side_transaction_time >= 6):
                            action = "SELL"
                            number_of_orders = len(sell_ids)
                            sell_price = ask_price - fee
                            price = sell_price
                            ids = sell_ids
                            volumes = sell_volumes
                            volumes_filled = volume_filled_sells
 
                            #delete sells then re-sell
                            re_order(s, number_of_orders, ids, volumes_filled, volumes, price, action)
                            sleep(SPEEDBUMP)
 
    #refresh the case time.
    tick, status = get_tick(s)

if __name__ == '__main__':
    signal.signal(signal.SIGINT, signal_handler)
    main()